# Multilingual Movie Database

The Internet might be the fastest growing phenomenon the world has seen. Cheap Internet-enabled mobile phones have accelerated this growth even further, and by some estimates, 40% of the world today has access to the Internet. Any web applications that we develop can truly be global. However, English users make up only around 30% of the Internet population. If your website is in English only, you are missing out on a huge audience.

To fix this, many efforts have been made in recent years to make websites accessible to non-English users as well. Django itself includes reliable methods to translate the content of your site into multiple languages.

However, translating the content is just the first part of the process. Language is not the only thing that is different between different parts of the world. Currency codes, time zones, and number formats are just a few examples. Adapting these to the location of your user is called 
**localization**. You will often see this abbreviated as **l10n**. That's the first l of localization, then a numeric 10, followed by the last n
. The 10 refers to the number of characters between the two! You may also come across the term internationalization (**i18n**). Internationalization is making sure that your application works across multiple regions without errors. For instance, making sure any inputs that you accept from the user can be in a variety of languages, and not just the one you developed the app in.

In this chapter, we will make an application inspired by the amazingly useful **IMDB** (**Internet Movie Database**) website. If you have never heard of it, it's a web application that provides a lot of information about movies, both old and new. We will be creating an application that provides some very basic features similar to IMDB. As our application is multilingual (which IMDB is as well by the way), I will refer to it as the 
**Multilingual Movie Database** (**MMDB**).

The code pack for this chapter includes a working non-localized copy of the application. Our job is to add localization and internationalization to it so that it works well for our users in France.

## Requirements

Let's take a look at what we want to achieve by the end of this chapter:

* Getting an overview of all the features provided by Django to allow 
localization
 
* Translating the contents of the site into French
 
* Giving users the ability to choose which language they want to use the site in
 
* Persisting the language preference of the user across multiple visits
 
* Translating the content of models

Before we start, there is one thing I'd like to mention. As we are learning this stuff for the first time, we will start with an already existing Django application. However, our application is very small compared to most real-world projects. For larger applications, it is usually more difficult to add localization after finishing the project

It is always a good idea to think about localization requirements when starting the project and then incorporating those features while developing the application for the first time, rather than doing so at a later stage when the application has been developed.

## Getting the project up and running

As always, once you have downloaded the code drop, unzip it. Then, create a new virtual environment for this project and install Django. Finally, activate it and run the migrate command from in the project root. This should set up the database for the project and get you to a point where you can start the application. Now you need to create a new super user so that you can add some test data. From within the project root (with the virtual environment active), run the following command:

```sh
> python manage.py createsuperuser
```

Answer the questions and you'll have a new user. Now, run the application with the `runserver` command, then visit `http://127.0.0.1:8000/admin/`, and add a few movie detail objects to the database. Once you have added some test data, visit the home page for the application, and you should see something similar to the following screenshot:

![MMDB: Movies List](img/MMDB_MoviesList.png)

You should take some time to explore the app. You can view details for a particular movie on a page, as shown in the following screenshot:

![MMDB: Kung Fu Panda](img/MMDB_KungfuPanda.png)

Finally, you can click on the **Submit New Review** link to get to the following page and create a new review for the movie, as shown in the following screenshot:

![MMDB: Submit New Review for Kung Fu Panda](img/MMDB_SubmitNewReview.png)

That's our entire application. For the rest of the chapter, we'll be looking into how to add l10n and i18n to this project. We'll be making very few, if any, changes to the core product features.

## Translating our static content

The first thing that we want to do is translate all the static content on our site. This includes all the headlines, links, and form labels that you have seen in the preceding three screens. To translate strings that are used in templates, Django provides us with a `trans` template tag. Let's take a look at how to use it first in a simple context, and then I'll go into the details of how it works. This is a slightly long section as we will do a lot of things here that make up the foundation of the translation feature of Django.

>tips

>Don't be alarmed if you don't understand something. Just keep following the instructions. I'll go into a lot of depth about each step, but first I want to show you exactly how translations are done.

Open up `main/templates/movies_list.html`, and replace `Movies List` in the `h2` tag with the following:

```html
{% trans "Movies List" %}
```

Add the following `load` tag to the second line of the file, right after the `extends` tag:

```html
{% load i18n %}
```

That's all the changes that we need to make to the template for now. I'll be explaining what these two lines do in just a little bit, but first I want to complete the whole translation process so that you can look at the whole thing instead of just smaller parts.

Next, let's run this command from the project root:

```sh
> python manage.py makemessages -l fr 
CommandError: Unable to find a locale path to store translations for file main/__init__.py
```

If you run this command, you should also see the same error as I did, something being unable to find a locale path. We'll explain what the locale path is after we're done with the demo. For now, create a new folder called `locale` in the `main` folder and run the command again:

```sh
>mkdir main/locale
> python manage.py makemessages -l fr
processing locale fr
```

This time the command succeeds. If you look in the locale folder that you created, you should see that a whole new hierarchy of folders has been created underneath it. What the `makemessages`
command did was create a `django.po` at `main/locale/fr/ LC_MESSAGES/django.po` file. If you open this file, you should be able to figure out a bit about its purpose. The last three lines of the file should be as follows:

```
#: main/templates/movies_list.html:5
msgid "Movies List"
msgstr ""
```

Together with the path of this file (`locale/fr/LC_MESSAGES/django.po`) and these three lines, you should be getting the idea that this file will contain the translated French text for the string that we marked earlier with the `trans` tag. Anything you put in quotes next to `msgstr` is what will replace the original string in the French translation of the site.

I used Google Translate to translate the `Movies List` string, and it gave me the translation as Liste des films. Put this translation in the quotes next to msgstr. The last three lines of the `django.po` file should now match the following:

```
#: main/templates/movies_list.html:5
msgid "Movies List"
msgstr "Liste des films"
```

Next, run this command from the project root:

```
> python manage.py compilemessages -l fr
processing file django.po in /Users/asadjb/Programming/Personal/
DjangoBluePrints/mmdb/mmdb/main/locale/fr/LC_MESSAGES
```

If you were to look in the `LC_MESSAGES` folder now, you should see that a new `django.mo` file has been created. This is the compiled version of our `django.po` file, the one that we put the translated strings into. For performance, Django translations require the file to be compiled into a binary format before it can pick up translations for strings

Next, open up `mmdb/settings.py` and find the `MIDDLEWARE_CLASSES` list. Edit it so that the `django.middleware.locale.LocaleMiddleware` string appears between the already installed 
S`essionMiddleware` and `CommonMiddleware`. The position is important. The list should now look as follows:

```
MIDDLEWARE_CLASSES = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.locale.LocaleMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.auth.middleware.SessionAuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]
```

Next, add a `LANGUAGES` variable to the settings file and give it the following value:

```
LANGUAGES = (
    ('en', 'English'),
    ('fr', 'French')
)
```

By default, Django supports a much longer list of languages. For our project, we want to restrict the user to only these two options. That is what the `LANGUAGES` list does.

The last step is to modify the `mmdb/urls.py` file. First, import `i18n_patterns` from `django.conf.urls.i18n`. Next, change the `urlpatterns` variable so that the `i18n_patterns` function wraps all our URL definitions, as shown in the following code:

```
urlpatterns = i18n_patterns(
url(r'^$', MoviesListView.as_view(), name='movies-list'),
url(r'^movie/(?P<pk>\d+)/$', MovieDetailsView.as_view(), name='movie-
details'),
url(r'^movie/(?P<movie_pk>\d+)/review/$', NewReviewView.as_view(), 
name='new-review'),
url(r'^admin/', admin.site.urls),
)
```   

With this done, let's test and see what our hard work got us. First, open up `http://127.0.0.1:8000`. You should see the same home page as before, but if you pay attention to the address bar, you will notice that the browser is at `http://127.0.0.1:8000/en/` instead of what we entered. We will look at the details of why this happens next, but in a nutshell, we opened the home page without specifying a language and Django redirected us to the default language for the site, which we specified earlier as English.

Change the URL to `http://127.0.0.1:8000/fr/` and you should see the same home page again, but this time, the `Movies List` text should be replaced by what we said was its French translation, as shown in the following screenshot:

![MMDB: Movies List in Fr](img/MMDB_MoviesListFr.png)

While all of this probably seems like a lot of work to translate a single sentence, remember that you only need to do this once. Let's see how easy it is to translate something else now that the foundation is there. Let's translate the word `Stars` to its French translation, Etoiles. Open up `main/templates/movies_list.htm`l and replace the word `Stars` with the following:

```html
{% trans "Stars" %}
```

Next, run the `makemessages` command:

```sh
> python manage.py makemessages -l fr
```

Open the `main/locale/fr/LC_MESSAGES/django.po` file. You should see a new section for the `Star` string that we marked for translation. Add the translation (`Étoile`) and save the file. Finally, run the `compilemessages` command:

```sh
> python manage.py compilemessages
```

Open the French language home page again by visiting `http://127.0.0.1:8000/fr/`. You will see that the word `Stars` has been replaced by its French translation. The effort involved was minimal. The workflow that you just followed: marking one or more strings for translation, making messages, translating the new strings, and finally running `compilemessages` is one followed by most Django developers when they are translating a project. Most of the effort involved with getting a site translation ready is all the work that we did beforehand. Let's take a closer look at what exactly we have done to get our web application translatable.

## How did all that work?

As I promised at the start of the previous section, after seeing Django translations in action, we will now take a deeper look into all the steps we followed to get to this point and what each of these steps did.

The first thing that we did was load the i18n template tags library, which provides us with a variety of template tags to translate content in the template. The most important, and probably the one that you will use the most, is the `trans` tag. The `trans` tag accepts a string argument and, depending on the language that is active, outputs the correct translation for that string. If the translation cannot be found, the original string is output instead.

Almost any string that you write in your templates will end up being wrapped by the `trans` tag and then later translated to the various languages that your web application is available in. There are certain situations in which the `trans` tag is not usable. For instance, if you have to add the value of some context variable to the translated string, the `trans` tag can't do this. For these cases, we need to use the block translation tag, blocktrans. We won't be needing it in our application, but you can read about it in the Django documentation at <https://docs.djangoproject.com/es/stable/topics/i18n/translation/#blocktrans-template-tag>.

Our next step was to run the `make messages` command. Our first attempt didn't succeed, so we had to create a `locale` directory in our `application` folder. Having done that, we ran the command and it created a message file with the `.po` extension. What the command does is it goes over every file in your project and extracts strings that you have marked for translation. One way to mark a string is to use the `trans` tag to wrap it. There are other ways as well that we will look at later.

After the `make messages` command has extracted the strings, it needs to create files and store the extracted strings in these files. There is a set of rules that Django follows when figuring out which file each extracted string goes to. For strings extracted from the files of an app, Django first tries to find a `locale` directory in the folder for that app. If it finds the folder, it creates the appropriate hierarchy underneath it (the `fr/LC_MESSAGES` directories) and
places the messages file there.

If the `locale` folder is not found, Django looks at the value of the `LOCALE_PATHS` settings variable. This should be a list of directory locations. Django selects the first directory from this list of paths and puts the messages file there. In our case, we didn't have the `LOCALE_PATHS` setup, which is why Django raised an error, not finding a locale directory in our main application folder.

Let's talk a bit about the format of the messages file. Here is what our messages file looks like right now:

```
# SOME DESCRIPTIVE TITLE.
# Copyright (C) YEAR THE PACKAGE'S COPYRIGHT HOLDER
# This file is distributed under the same license as the PACKAGE 
package.
# FIRST AUTHOR <EMAIL@ADDRESS>, YEAR.
#
#, fuzzy
msgid ""
msgstr ""
"Project-Id-Version: PACKAGE VERSION\n"
"Report-Msgid-Bugs-To: \n"
"POT-Creation-Date: 2016-02-15 21:25+0000\n"
"PO-Revision-Date: YEAR-MO-DA HO:MI+ZONE\n"
"Last-Translator: FULL NAME <EMAIL@ADDRESS>\n"
"Language-Team: LANGUAGE <LL@li.org>\n"
"Language: \n"
"MIME-Version: 1.0\n"
"Content-Type: text/plain; charset=UTF-8\n"
"Content-Transfer-Encoding: 8bit\n"
"Plural-Forms: nplurals=2; plural=(n > 1);\n"
```

```
#: main/templates/movies_list.html:6
msgid "Movies List"
msgstr "Liste des films"
```

```
#: main/templates/movies_list.html:10
msgid "Stars"
msgstr "Étoile"
```

Lines starting with # are comments. Then, there is an empty pair of `msgid` and `msgstr`. This is followed by some metadata about this messages file. After that, we get the main part of the time. A messages file, ignoring the metadata and first pair (the one preceded by the fuzzy comment), is just a list of `msgid` and `msgstr` pairs. The `msgid` pairs is the string that you have marked for translation, and `msgstr` is the translation for that string. The usual method of translating an app is by first marking all the strings for translation, then generating the messages file, and finally providing it to the translator. The translator then returns the file to you with the translations filled in. The benefit of using a simple text file is that the translator doesn't need to use any special software. If he has access to a simple text editor, he can translate the messages file.

Once we have translated the strings in the messages file, we need to run the compile messages command before Django is able to use the translations. The compile command, as mentioned earlier, converts the text messages file to a binary file. The binary file format is much quicker to read from for Django, and in projects with hundreds or thousands of translatable strings, these performance benefits add up very quickly. The output of the compile messages file is a .mo file in the same folder as the .po file.

Once we have our translations done and compiled, we need to set up a few Django configurations. The first thing we do is add `LocaleMiddleware` to the list of middlewares that our application uses. The job of `LocaleMiddleware` is to allow users to select the language of the site based on a couple of request parameters. You can read the details of how the language is determined in the documentation at <https://docs.djangoproject.com/es/stable/topics/i18n/translation/#how-django-discovers-language-preference>. We will come back to it in a bit, discussing how it determines the language with an example.

We then needed to defined two settings variables, `LANGUAGES` and `LANGUAGE`. `LANGUAGE` was already defined in the code pack, so we only set the `LANGUAGES` variable. `LANGUAGES` is a list of language choices that Django can provide translations for the site in. By default, this is a huge list that includes all languages that Django can be translated into. However, for most projects, you want the user limited to a few languages to use the site in. By providing our own value for the `LANGUAGES` list, we ensure that Django doesn't serve pages for any languages other than the ones defined.

The `LANGAUGE` variable defines the default language to use. If you remember, when we opened the home page without any language code (`http://127.0.0.1:8000/`), the **English** language was selected by default. The `LANGUAGE` variable decides what the default language for the site is.

The next part of making the app translatable was to modify the `url.py` file. In place of the simple list of URL configuration elements, we wrapped our URL configurations inside of an `i18n_patterns` function. This function allows us to match URLs that have a language code prepended to them. For every request that comes in, this function tries to match the patterns that we wrapped in it after removing the language code from the URL path. It's a bit complicated to explain, so let's look at an example.

Let's say that we have the following URL pattern:

```
url(r'^example/$', View.as_view(), name='example')
```

This would match `DOMAIN.COM/example/`, but if we tried `DOMAIN.com/en/example/`, the pattern would not result in a match as the `/en/` part is not part of the regex. However, once we wrap it in 
`i18n_patterns`, it will match the second example. This is because the `i18n_patterns` function removes the language code and then tries to match the patterns that we wrapped in it.

In some applications, you don't want to have all the URLs matching with language prefixes. Some URLs, such as an API endpoint, don't change based on the language. In these cases, you can add together `i18n_patterns` and the normal list of URL patterns:

```
urlpatterns = i18n_patterns(url(r'^example/$', ExampleView.as_view(), 
name='example')) + [url(r'^api/$', ApiView.as_view(), name='api')]
```

This way, you can create applications that are a mix of translated and non-translated views.

Having added `i18n_urlpatterns`, we are done with all the configuration that Django needs for basic internationalization, and we can visit the pages that we have in the French language and see the translated versions.

The last thing that I have left to explain is LocaleMiddleware. The locale middleware is the part of Django that allows users to use the language code in the url to decide which language to use. Thus, even though it's `i18n_patterns` that matches patterns based on language codes, it's the middleware that activates the correct language for each request. Other than using the language prefix in the URL path, `LocaleMiddleware` provides you with a few other ways to select the language as well:

* A session variable
 
* A cookie value
 
* The Accept-Language header that the user's browser sends

* If all else fails, the default language from the `LANGUAGE` setting variable is used

This is an overview of how we adapted our application to be translatable. However, we're not done yet.

## Letting the user decide which language to use

While it's not a part of Django, almost all projects that are internationalized use this pattern; thus I think it's important that you are aware of it. Most sites that have multiple language options present the user with a menu to select which language they want to view the site in. Let's create that. Modify the `templates/base.html` template to match the following:

```html
{% load i18n %}

<html>
<head>
<meta http-equiv="content-type" content="text/html; charset=utf-8" />
<title>MMDB</title>
</head>
<body>
<h1>MMDB</h1>
<div>
<span>Select Language</span>
<ul>
            {% get_available_languages as available_languages %}
            {% for lang_code, lang_name in available_languages %}
            {% language lang_code %}<li><a href="{% url "movies-list" 
%}">{{ lang_name }}</a></li>{% endlanguage %}
            {% endfor %}
</ul>
</div>
    {% block content %}
    {% endblock %}
</body>
</html>
```

The new parts are highlighted. We first import the `i18n` template library. Then, we create a new 
`div` element to hold our list of language choices. Next, to get the language choices as part of the template, we use the `get_available_languages` template tag and assign the choices to the 
`available_languages` variable.

Next, we create a list of links based on the language choices. The return value  of `get_available_languages` is the tuple that we set in the settings file for the `LANGUAGE` variable.

In our list of links, we need some way to get URLs for each language. Again, Django shines here, with deep integration between the internationalization features and the rest of the framework. If you have internationalization active and reverse a URL, it automatically gets the correct language prefix.

However, we can't just do a reverse for the URL here because that would create URLs for the language that is currently active. Thus, our list of links to switch language would actually just point to the current language. Instead, we have to temporarily switch to the language we want to create a link for and then generate the URL. We do this using the `language` tag. Between the `language` tag, the language that we pass as a parameter is activated. Thus, our reversed URLs turn out exactly like we want them.

One last thing to note is the URL that we reverse. For our application, the `movies-list` URL is the home page, thus we reverse that. For most applications, you'll do the same and reverse the home page URL so that switching the language takes the user to the home page in the specified language.

>tips

>There are advanced ways by which you can keep the user on the current page and still switch the language. One is to generate the link on each page instead of `base.html`, as we have done here. This way, as you know which URL the template will be rendered for, you can reverse the appropriate URL. However, this has the drawback of requiring you to repeat yourself a lot. You can search on Google for Django reverse current URL in another language and get some other suggestions. I still haven't found a good one to use, but you can decide if you think one of the suggested options fits your needs.

nce you have made the changes, open the movies list page again by visiting `http://127.0.0.1:8000/en/`, and you should now see the language switcher links at the top. Refer to the following screenshot:

![MMDB: Select Language](img/MMDB_SelectLanguage.png)

You can try switching the languages and see the change reflect immediately in the strings on the page.

## Persisting the user choice

Let's try an experiment. Switch the language to French, and then close the browser window. Open the browser again and visit `http://127.0.0.1:8000/`. Note the absence of a language prefix in the URL. You will be redirected to the English language of the site. Wouldn't it be nice if, once you selected which language to use, it would persist across visits?

Django provides such a feature out of the box; you just have to add a few bits of code to use it. If you remember the list of steps that `LocaleMiddleware` takes to determine the language for the current request, the second step—after looking at the URL for a prefix—was to look at the session. If we can put the language choice in the session dictionary, Django will automatically choose the correct language for the user on subsequent visits.

What is the correct place to put the piece of code that updates the session dictionary? If you think about it, every time the user changes their language selection, we redirect them to the home page. As they will always visit the home page when their language preference changes, let's put our code there. Modify `MoviesListView` to match the following code:

```
class MoviesListView(ListView):
    model = MovieDetails
    template_name = 'movies_list.html'
    def get(self, request, *args, **kwargs):
        current_language = get_language()
        request.session[LANGUAGE_SESSION_KEY] = current_language
        return super(MoviesListView, self).get(request, *args, 
**kwargs)
```

You will also need to import `get_language` and `LANGUAGE_SESSION_KEY`. Put this at the top of `main/views.py`:

```
from django.utils.translation import LANGUAGE_SESSION_KEY
from django.utils.translation import get_language
```

Now, visit the site again and change your language to French. Next, close the browser window and open it up again. Open `http://127.0.0.1:8000/` and be careful not to put the language prefix in the URL, and you should be redirected to the French language page.

Let's look at what's happening here. In the absence of a language code in the URL, `LocaleMiddleware` looks at the session to see if the key that holds the language selection has any value. If it does, the middleware sets that as the language for the request. We put the language selection of the user in the session by first getting the language that is currently active using the `get_language` method and then putting it in the session. The key name used by the middleware is stored in the `LANGUAGE_SESSION_KEY` constant, so we use this to set the language selection.

With the session set properly, the next time your user visits the site without a language prefix, the middleware finds their choice in the session and uses that.

## Translating our models

The last thing that we want to look at is how to translate our model data. Open the site and change to the French language. Your home page should be similar to the following screenshot:

![MMDB: Select Language](img/MMDB_SelectLanguage2.png)

You will notice that even though the static content—the one that we put in the template ourselves—is translated, the dynamic names of the movies are not. While this is acceptable for some sites, your model data should be translated as well in order to be truly internationalized. Django by default does not have any built-in method to achieve this, but it's pretty easy.

>tips

>What I'm about to show you is something that the Django 
modeltranslation
 library already provides. I have used it in a 
large scale project and it works pretty well, so if you want to skip this 
section, you can just go ahead and use the library. However, it's nice to 
have an overview of how you can achieve it without any external help.
You can find the library at 
<https://github.com/deschler/django-modeltranslation>.

What we need is some way to store more than one language for each of the text fields in our models. You could come up with a scheme where you store both the English and French translation of the string in the same field using some separator, and then separate the two when displaying the model.

Another way to achieve the same result would be to add an extra field for each language. For our current example, that would mean adding an extra field for each field that we want to translate.

Both approaches have their pros and cons. The first one is difficult to maintain; as you add more than just one language to translate to, the data format becomes difficult to maintain.

The second approach adds database fields, something that may not always be possible. Plus, it requires a fundamental change in how to access the data. However, if you have the option, I always advice going with the option that results in cleaner and easy-to-understand code, which in this case means adding extra fields per language.

For our MovieDetails model, this means having an extra field each for the title and description fields to store the French translation. Edit your `main/models.py` file so that the `MovieDetails` model matches the following code:

```
class MovieDetails(models.Model):
    title = models.CharField(max_length=500)
    title_fr = models.CharField(max_length=500)
    
    description = models.TextField()
    description_fr = models.TextField()
    
    stars = models.PositiveSmallIntegerField()
    
    def __str__(self):
        return self.title
```

Next, create and run the migrations to add these new fields to the database:

```
> python manage.py makemigrations
You are trying to add a non-nullable field 'description_fr' to 
moviedetails without a default; we can't do that (the database needs 
something to populate existing rows).
Please select a fix:
 1) Provide a one-off default now (will be set on all existing rows)
 2) Quit, and let me add a default in models.py
Select an option: 1
Please enter the default value now, as valid Python
The datetime and django.utils.timezone modules are available, so you can 
do e.g. timezone.now()
>>> ''
You are trying to add a non-nullable field 'title_fr' to moviedetails 
without a default; we can't do that (the database needs something to 
populate existing rows).
Please select a fix:
 1) Provide a one-off default now (will be set on all existing rows)
 2) Quit, and let me add a default in models.py
Select an option: 1
Please enter the default value now, as valid Python
The datetime and django.utils.timezone modules are available, so you can 
do e.g. timezone.now()
>>> ''
Migrations for 'main':
  0002_auto_20160216_2300.py:
    - Add field description_fr to moviedetails
    - Add field title_fr to moviedetails
    - Alter field movie on moviereview
```

As you can see in the preceding CLI session, when I created the migrations, I was asked to provided a default value for the new fields. I just entered the empty string. We can fix the value later from the admin.

Finally, run the new migration:

```
> python manage.py migrate
Operations to perform:
  Apply all migrations: admin, contenttypes, main, auth, sessions
Running migrations:
  Rendering model states... DONE
  Applying main.0002_auto_20160216_2300... OK
```

Once this is done, open up the admin and see the edit page for one of the objects you have in your database. It should seem similar to the following screenshot:

![MMDB: Django administration](img/MMDB_Administration.png)

In the preceding screenshot, you can see that two new fields have been added to the admin. I translated the English value for these fields using Google Translate and filled in the French language field values. Click on **Save**. Now, our models have the French language data alongside the English values; but how to display them?

You could put a couple of `if/else` conditions in your template code to decide which language field to use. However, that gets messy quickly. Right now, our model only has two fields that are translated, but imagine a model with 10 such fields. How many conditions would you have? We're only talking about supporting one language. Finally, we only have two templates that need to be modified, the list view and detail view. In a real-world, more complicated application, your models could be used from a hundred different places. The `if/else` approach gets too difficult to maintain very quickly.

Instead, what we will do is give our model methods to intelligently return to us the correct value for a field, depending on the current language. Let's modify our `main/models.py` file again. First, import the `get_language` method at the top:

```
from django.utils.translation import get_language
```

Next, modify the `MovieDetails` model again and add these three new methods (highlighted in the code):

```
class MovieDetails(models.Model):
    title = models.CharField(max_length=500)
    title_fr = models.CharField(max_length=500)
    
    description = models.TextField()
    description_fr = models.TextField()    
    
    stars = models.PositiveSmallIntegerField()    
    
    def get_title(self):
        return self._get_translated_field('title')        
        
    def get_description(self):
        return self._get_translated_field('description')        
        
    def _get_translated_field(self, field_name):
        original_field_name = field_name        
        lang_code = get_language()        
        
        if lang_code != 'en':
            field_name = '{}_{}'.format(field_name, lang_code)
        field_value = getattr(self, field_name)        
        
        if field_value:
            return field_value
        else:
            return getattr(self, original_field_name)            

    def __str__(self):
        return self.title
```

There's nothing Django-specific in the new methods. The main work-horse is the `_get_translated_field` method. Given a field name, it looks at the current language, and if the language is something other than English, appends the language code to the field name. It then gets the value for the new field name from the object. If the value is empty because we didn't translate the field, it follows the Django convention and just returns the value of the original non-translated field.

Now, modify `main/templates/movies_list.html` to use these new methods:

```
{% extends "base.html" %}
{% load i18n %}

{% block content %}
<h2>{% trans "Movies List" %}</h2>
<ul>
    {% for movie in object_list %}
<li><a href="{% url 'movie-details' pk=movie.pk %}">{{ 
movie.get_title
}}</a> | {{ movie.stars }} {% trans "Stars" %}</li>
    {% endfor %}
</ul>
{% endblock %}
```

The only change here is that instead of using the value of `movie.title` directly, we use 
`movie.get_title`. This is the one major drawback of this approach. Now everywhere in your project where you need the title or description values, you'll have to use the `get_title` and 
`get_description` methods instead of just using the field values directly.

>notes

>The same goes for saving the fields. You'll have to figure out which 
field name to write to, depending on the active language. While 
neither of these are complicated, they do add some discomfort to the 
whole process. However, that's the price you pay for this power.
The `django-modeltranslation` package that I mentioned before 
has a good solution to this problem. It uses code in the model to 
automatically decides which language to return whenever you access 
any field. So, instead of using 
`obj.get_title()`, you would write `obj.title`
 and get the correct field for the currently activated 
language. For your projects, you might want to look into this. I didn't 
use this in this chapter because I wanted to give you a way to work 
with basic Django and show you one possible way to do things 
yourself instead of relying on third-party libraries.

Open up the French version of the site again and you should see that the one object we translated should have the translated version of the title, whereas the others will just show the non-translated versions:

![MMDB: Movies List in France](img/MMDB_MoviesListFr2.png)

Doing the same for the details template should be simple and is left up to you!

## Summary

While this was a somewhat small chapter, we looked at information that will come in handy throughout your web development career. While not all of the sites that you develop will need to be translated into multiple languages, some of the most important ones will. When you get a chance to work on such a project, you will have the information on how to go about creating a truly internationalized web application.

We have only scratched the surface of what is possible with Django internationalization and localization. When you start a project that requires these, be sure to check out the documentation as well.

We are now done with simple Django-only applications. Having learnt the basics of Django, the next chapter will have us working on a more complicated web application—one that involves search using the amazingly powerful Elasticsearch!